In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

INIT_LR = 1e-4 #learning rate
EPOCHS = 20 # number of itertaion
BS = 32 


# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images

#pathname = "/content/drive/MyDrive/Tanweer Internships & Jobs/SWoC/Face-Mask-Detection-master/dataset"
pathname = "C:\\Users\\NIC-PC\\Documents\\AI Codes\\Face-Mask-Detection\\dataset"

print("[INFO] loading images...")
imagePaths = list(paths.list_images(pathname))
data = []
labels = []

# loop over the image paths
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]

	# load the input image (224x224) and preprocess it
	image = load_img(imagePath, target_size=(224, 224))
	image = img_to_array(image)
	image = preprocess_input(image)

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)

# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["accuracy"])

# train the head of the network
print("[INFO] training head...")
H = model.fit(aug.flow(trainX, trainY, batch_size=BS),steps_per_epoch=len(trainX) // BS,validation_data=(testX, testY),validation_steps=len(testX) // BS,epochs=EPOCHS)


[INFO] loading images...


C:\ProgramData\Anaconda3\lib\site-packages\PIL\Image.py:951: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


[INFO] compiling model...
[INFO] training head...
Epoch 1/20
102/102 [==============================] - 468s 4s/step - loss: 0.6898 - accuracy: 0.6702 - val_loss: 0.1598 - val_accuracy: 0.9780
Epoch 2/20
102/102 [==============================] - 347s 3s/step - loss: 0.1827 - accuracy: 0.9539 - val_loss: 0.0885 - val_accuracy: 0.9866
Epoch 3/20
102/102 [==============================] - 316s 3s/step - loss: 0.1162 - accuracy: 0.9708 - val_loss: 0.0661 - val_accuracy: 0.9866
Epoch 4/20
102/102 [==============================] - 289s 3s/step - loss: 0.1011 - accuracy: 0.9688 - val_loss: 0.0576 - val_accuracy: 0.9866
Epoch 5/20
102/102 [==============================] - 305s 3s/step - loss: 0.0871 - accuracy: 0.9775 - val_loss: 0.0545 - val_accuracy: 0.9866
Epoch 6/20
102/102 [==============================] - 367s 4s/step - loss: 0.0722 - accuracy: 0.9824 - val_loss: 0.0488 - val_accuracy: 0.9878
Epoch 7/20
102/102 [==============================] - 371s 4s/step - loss: 0.0589 - accuracy